In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

In [2]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo a una tabla:

In [3]:
%%sql
create or replace table competencia_02 as 
select 
    *
from read_csv_auto('C:/Users/tgian/DMEyF/datasets/competencia_02_crudo.csv')

,Success


In [5]:
# %%sql
# create or replace table targets as
# with periodos as (
#     select distinct foto_mes from competencia_02
# ), clientes as (
#     select distinct numero_de_cliente from competencia_02
# ), todo as (
#     select numero_de_cliente, foto_mes from clientes cross join periodos
# ), clase_ternaria as (
#     select
#         t.numero_de_cliente
#         , t.foto_mes
#         , if(c.numero_de_cliente is null, 0, 1) as mes_0
#         , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
#         , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
#         , null as clase_ternaria -- AGREGAR LÓGICA
#     from todo t
#     left join competencia_02 c using (numero_de_cliente, foto_mes)
# ) select
#   foto_mes
#   , numero_de_cliente
#   , clase_ternaria
# from clase_ternaria where mes_0 = 1

,Success


Código compartido por un compañero:

In [5]:
%%sql
create or replace table final AS
WITH aux AS  (
    SELECT numero_de_cliente, foto_mes,
    (foto_mes//100-2019)*12+foto_mes%100 as mes,
    IF(foto_mes = (select max(foto_mes) from competencia_02) , -1, lead(mes,1,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+1',
    IF(foto_mes = (select max(foto_mes) from competencia_02) OR foto_mes = ((select max(foto_mes) from competencia_02)-1), -1,lead(mes,2,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+2',
    FROM
       competencia_02
    ORDER BY numero_de_cliente, foto_mes)
SELECT *,
CASE
    -- Condiciones de borde:
    WHEN ("mes+1"==-1) THEN 'CONTINUA'
    WHEN ("mes+2"==-1) THEN 'CONTINUA'
    -- Si mes siguiente no es consecutivo o no existe:
    WHEN ("mes"+1 != "mes+1") THEN 'BAJA+1'
    WHEN ("mes"+2 != "mes+2") THEN 'BAJA+2'
    -- si proximos 2 meses consecutivos activos:
    ELSE 'CONTINUA' END AS "clase_ternaria"
FROM aux
LEFT JOIN competencia_02 c USING (numero_de_cliente, foto_mes);

,Success


### Chequeos:

In [16]:

%%sql
SELECT clase_ternaria, COUNT(*) AS count
FROM final
GROUP BY clase_ternaria


,clase_ternaria,count
0,BAJA+2,18987
1,BAJA+1,18843
2,CONTINUA,4524980


In [8]:
%%sql
select *
from final
limit 10

,numero_de_cliente,foto_mes,mes,mes+1,mes+2,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,168102278,202003,15,16,17,1,0,1,32,14,...,6,0.0,-10523.95,0.0,280,0.0,0.0,0.0,516.12,CONTINUA
1,168102278,202004,16,17,18,1,0,1,32,15,...,1,0.0,-10523.95,0.0,310,0.0,0.0,0.0,0.00,CONTINUA
2,168102278,202005,17,18,19,1,0,1,32,16,...,4,0.0,NaN,NaN,341,NaN,NaN,NaN,0.00,CONTINUA
3,168102278,202006,18,19,20,0,0,0,32,17,...,-1,0.0,NaN,NaN,371,NaN,NaN,NaN,0.00,CONTINUA
4,168102278,202007,19,20,21,1,0,1,32,18,...,2,0.0,NaN,NaN,402,NaN,NaN,NaN,0.00,CONTINUA
5,168102278,202008,20,21,22,1,0,1,32,19,...,5,0.0,NaN,NaN,433,NaN,NaN,NaN,0.00,CONTINUA
6,168102278,202010,22,23,24,1,0,0,32,21,...,3,0.0,NaN,NaN,494,NaN,NaN,NaN,0.00,CONTINUA
7,168102278,202011,23,24,25,1,0,0,32,22,...,5,0.0,NaN,NaN,524,NaN,NaN,NaN,0.00,CONTINUA
8,168102278,202101,25,26,27,1,0,0,32,24,...,4,0.0,NaN,NaN,586,NaN,NaN,NaN,0.00,CONTINUA
9,168102278,202102,26,27,28,1,0,0,32,25,...,-3,0.0,NaN,NaN,614,NaN,NaN,NaN,0.00,CONTINUA


Guardo el csv con la clase ternaria:

In [17]:
%%sql 
COPY final TO 'C:/Users/tgian/DMEyF/datasets/competencia_02.csv' (FORMAT CSV, HEADER)

,Success


### Resolución

In [10]:
%%sql
create or replace table final AS
WITH aux AS  (
    SELECT numero_de_cliente, foto_mes,
    (foto_mes//100-2019)*12+foto_mes%100 as mes,
    IF(foto_mes = (select max(foto_mes) from competencia_02) , -1, lead(mes,1,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+1',
    IF(foto_mes = (select max(foto_mes) from competencia_02) OR foto_mes = ((select max(foto_mes) from competencia_02)-1), -1,lead(mes,2,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+2',
    FROM
       competencia_02
    ORDER BY numero_de_cliente, foto_mes)
SELECT *,
CASE
    -- Condiciones de borde:
    WHEN ("mes+1"==-1) THEN 'CONTINUA'
    WHEN ("mes+2"==-1) THEN 'CONTINUA'
    -- Si mes siguiente no es consecutivo o no existe:
    WHEN ("mes"+1 != "mes+1") THEN 'BAJA+1'
    WHEN ("mes"+2 != "mes+2") THEN 'BAJA+2'
    -- si proximos 2 meses consecutivos activos:
    ELSE 'CONTINUA' END AS "clase_ternaria"
FROM aux
LEFT JOIN competencia_02 c USING (numero_de_cliente, foto_mes);

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.CatalogException) Catalog Error: Table with name competencia_02 does not exist!
Did you mean "information_schema.columns"?
[SQL: create or replace table final AS
WITH aux AS  (
    SELECT numero_de_cliente, foto_mes,
    (foto_mes//100-2019)*12+foto_mes%100 as mes,
    IF(foto_mes = (select max(foto_mes) from competencia_02) , -1, lead(mes,1,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+1',
    IF(foto_mes = (select max(foto_mes) from competencia_02) OR foto_mes = ((select max(foto_mes) from competencia_02)-1), -1,lead(mes,2,0) OVER (PARTITION BY "numero_de_cliente" ORDER BY numero_de_cliente, mes)) AS 'mes+2',
    FROM
       competencia_02
    ORDER BY numero_de_cliente, foto_mes)
SELECT *,
CASE
    -- Condiciones de borde:
   

In [4]:
%%sql
ALTER TABLE competencia_01
ADD COLUMN clase_ternaria VARCHAR(255);

UPDATE competencia_01 AS t1
SET clase_ternaria = 
  CASE
    -- Rule 1: 'BAJA+1'
    WHEN foto_mes = '202103' AND numero_de_cliente NOT IN (
      SELECT numero_de_cliente
      FROM competencia_01
      WHERE foto_mes IN ('202104', '202105')
    )
    THEN 'BAJA+1'
    
    -- Rule 2: 'BAJA+2'
    WHEN foto_mes = '202103' AND numero_de_cliente IN (
      SELECT numero_de_cliente
      FROM competencia_01
      WHERE foto_mes = '202104'
    )
    AND numero_de_cliente NOT IN (
      SELECT numero_de_cliente
      FROM competencia_01
      WHERE foto_mes = '202105'
    )
    THEN 'BAJA+2'
    
    -- Rule 3: 'CONTINUA'
    WHEN foto_mes = '202103'
    AND numero_de_cliente IN (
      SELECT numero_de_cliente
      FROM competencia_01
      WHERE foto_mes = '202104'
    )
    AND numero_de_cliente IN (
      SELECT numero_de_cliente
      FROM competencia_01
      WHERE foto_mes = '202105'
    )
    THEN 'CONTINUA'
    
    -- Rule 4: NULL
    ELSE NULL
  END;

,Success


In [9]:
# Guardo el archivo:
%sql COPY competencia_01 TO 'C:/Users/tgian/DMEyF/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

,Success


In [102]:
# %%sql
# ALTER TABLE competencia_01
# ADD COLUMN clase_ternaria VARCHAR(255);

# UPDATE competencia_01
# SET clase_ternaria = 
#     CASE
#         WHEN numero_de_cliente NOT IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202103) THEN NULL
#         WHEN numero_de_cliente IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202103) AND numero_de_cliente IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202104) AND numero_de_cliente IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202105) THEN 'CONTINUA'
#         WHEN numero_de_cliente IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202103) AND numero_de_cliente IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202104) AND numero_de_cliente NOT IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202105) THEN 'BAJA+2'
#         WHEN numero_de_cliente IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202103) AND numero_de_cliente NOT IN (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202104) THEN 'BAJA+1'
#         ELSE NULL
#     END;

,Success


In [ ]:
# %%sql
# WITH clients_202103 AS (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202103),
# clients_202104 AS (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202104),
# clients_202105 AS (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202105)
# SELECT *,
#     CASE
#         WHEN numero_de_cliente IN (SELECT * FROM clients_202103) AND numero_de_cliente IN (SELECT * FROM clients_202104)AND numero_de_cliente IN (SELECT * FROM clients_202105) THEN 'CONTINUA'
#         WHEN numero_de_cliente IN (SELECT * FROM clients_202103) AND numero_de_cliente IN (SELECT * FROM clients_202104) AND numero_de_cliente NOT IN (SELECT * FROM clients_202105) THEN 'BAJA+1'
#         WHEN numero_de_cliente IN (SELECT * FROM clients_202103) AND numero_de_cliente NOT IN (SELECT * FROM clients_202104) THEN 'BAJA+2'
#         ELSE NULL
#     END AS clase_ternaria
# FROM competencia_01

In [106]:
%%sql 
SELECT foto_mes, count(*) as cantidad 
FROM competencia_01 
GROUP BY foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


## Chequeo:

In [9]:
data = pd.read_csv(r"C:\Users\tgian\DMEyF\datasets\competencia_02.csv")

In [13]:
data.foto_mes.value_counts()

foto_mes
202107    164682
202106    164420
202105    164102
202104    163637
202103    163324
202102    162364
202101    161711
202012    161133
202011    160493
202010    159477
202009    158031
202008    156667
202007    155348
202006    153386
202005    150874
202004    149484
202003    148989
202002    146851
202001    143762
201912    140468
201911    138438
201910    136515
201909    134267
201908    132615
201907    130551
201906    129181
201905    127683
201904    127059
201903    126536
201902    125898
201901    124864
Name: count, dtype: int64

In [25]:
%%sql 
SELECT clase_ternaria, count(*) as cantidad 
FROM competencia_02 
GROUP BY clase_ternaria

,clase_ternaria,cantidad
0,NaN,4562810


In [16]:
%%sql 
SELECT clase_ternaria, count(*) as cantidad 
FROM competencia_01
WHERE  foto_mes = 202105
GROUP BY clase_ternaria

,clase_ternaria,cantidad
0,None,4562810


In [59]:
%sql SELECT * FROM competencia_01

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7.0,0.00,-114954.00,0.00,5938.0,101050.66,68.0,0.0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1.0,0.00,-40330.15,17.59,4089.0,26834.09,7.0,0.0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21.0,4692.00,-1173.00,0.00,7829.0,1651.36,3.0,0.0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1.0,0.00,-15988.67,0.00,7580.0,30025.29,11.0,0.0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21.0,380616.14,-97383.25,0.00,7827.0,359610.70,31.0,0.0,15600.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491058,186790659,202105,0,0,0,35,1,0.00,0.00,0.00,...,NaN,5.0,0.00,NaN,NaN,28.0,NaN,NaN,NaN,0.00
491059,186792315,202105,1,0,0,37,1,51.81,51.81,0.00,...,NaN,5.0,0.00,NaN,NaN,25.0,NaN,NaN,NaN,0.00
491060,186795526,202105,1,0,0,47,1,327.75,327.75,0.00,...,NaN,5.0,0.00,NaN,NaN,21.0,NaN,NaN,NaN,0.00
491061,186798379,202105,0,0,0,32,1,0.00,0.00,0.00,...,NaN,5.0,0.00,NaN,NaN,11.0,NaN,NaN,NaN,0.00


### *Pruebas*

Recordar que __%sql__ se usa para tirar la query en una sola línea, y __%%sql__, para tirarla en multi-líneas (formato convencional para hacer queries)

In [4]:
# Prueba 1
%sql select * from competencia_01 where numero_de_cliente = 186836338

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,186836338,202105,0,0,0,33,1,30.73,30.73,0.0,...,NaN,NaN,0.0,NaN,NaN,1,NaN,NaN,NaN,0.0


In [5]:
%%sql 
select * 
from competencia_01 
where numero_de_cliente = 186836338

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,186836338,202105,0,0,0,33,1,30.73,30.73,0.0,...,NaN,NaN,0.0,NaN,NaN,1,NaN,NaN,NaN,0.0


In [6]:
# Prueba 2
# cuenta cuantos casos hay en cada foto_mes y lo guarda en un campo llamado cantidad
%sql SELECT foto_mes, count(*) as cantidad FROM competencia_01 GROUP BY foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


In [7]:
%%sql 
SELECT foto_mes, count(*) as cantidad 
FROM competencia_01 
GROUP BY foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


In [49]:
%%sql 
SELECT DISTINCT foto_mes, count(*)
FROM competencia_01
GROUP BY foto_mes

,foto_mes,count_star()
0,202103,163324
1,202104,163637
2,202105,164102


In [14]:
%%sql
SELECT  *
FROM competencia_01
WHERE foto_mes = 202105

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202105,1,0,0,50,199,15510.89,153502.25,2699.83,...,0.0,12.0,87975.0,-50368.62,0.00,5999.0,114271.45,83.0,0.0,65066.31
1,29184630,202105,1,0,0,59,324,-393.25,62747.12,4464.30,...,0.0,5.0,0.0,-67978.28,17.59,4150.0,41404.61,9.0,0.0,5653.86
2,29185433,202105,1,0,0,68,270,1153.64,21758.23,610.39,...,0.0,26.0,4340.1,-4692.00,0.00,7890.0,1529.02,2.0,0.0,1372.41
3,29185587,202105,1,0,0,80,324,12323.31,56238.98,10364.73,...,0.0,5.0,0.0,-30151.02,0.00,7641.0,9672.92,8.0,0.0,985.32
4,29185646,202105,1,0,0,60,259,-3016.59,25094.10,1473.47,...,0.0,26.0,243671.6,-172084.36,23.40,7888.0,228283.93,33.0,0.0,10815.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164097,186790659,202105,0,0,0,35,1,0.00,0.00,0.00,...,NaN,5.0,0.0,NaN,NaN,28.0,NaN,NaN,NaN,0.00
164098,186792315,202105,1,0,0,37,1,51.81,51.81,0.00,...,NaN,5.0,0.0,NaN,NaN,25.0,NaN,NaN,NaN,0.00
164099,186795526,202105,1,0,0,47,1,327.75,327.75,0.00,...,NaN,5.0,0.0,NaN,NaN,21.0,NaN,NaN,NaN,0.00
164100,186798379,202105,0,0,0,32,1,0.00,0.00,0.00,...,NaN,5.0,0.0,NaN,NaN,11.0,NaN,NaN,NaN,0.00


In [36]:
%%sql
WITH clients_202104 AS (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202104),
clients_202105 AS (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202105)
SELECT *
FROM competencia_01 c
WHERE c.numero_de_cliente IN (SELECT * FROM clients_202104)

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7.0,0.00,-114954.00,0.00,5938.0,101050.66,68.0,0.0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1.0,0.00,-40330.15,17.59,4089.0,26834.09,7.0,0.0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21.0,4692.00,-1173.00,0.00,7829.0,1651.36,3.0,0.0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1.0,0.00,-15988.67,0.00,7580.0,30025.29,11.0,0.0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21.0,380616.14,-97383.25,0.00,7827.0,359610.70,31.0,0.0,15600.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488574,186462981,202105,1,0,0,60,2,1023.55,1023.55,1157.93,...,0.0,5.0,0.00,0.00,0.00,50.0,5043.90,2.0,0.0,1137.81
488575,186518258,202105,1,0,0,46,2,2906.64,3331.07,308.68,...,NaN,5.0,0.00,NaN,NaN,32.0,NaN,NaN,NaN,0.00
488576,186519915,202105,1,0,0,26,2,120.52,120.52,116.12,...,0.0,5.0,0.00,0.00,0.00,35.0,0.00,0.0,0.0,82.11
488577,186523058,202105,1,0,0,22,2,-6481.85,-6481.85,96.83,...,0.0,5.0,0.00,0.00,0.00,35.0,0.00,0.0,0.0,0.00


In [51]:
%%sql
WITH clients_202103 AS (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202103),
clients_202104 AS (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202104),
clients_202105 AS (SELECT numero_de_cliente FROM competencia_01 WHERE foto_mes = 202105)
SELECT *
FROM clients_202103 c_03
INNER JOIN clients_202104 c_04 ON c_03.numero_de_cliente = c_04.numero_de_cliente
INNER JOIN clients_202105 c_05 ON c_03.numero_de_cliente = c_05.numero_de_cliente

,numero_de_cliente,numero_de_cliente_2,numero_de_cliente_3
0,29183981,29183981,29183981
1,29184630,29184630,29184630
2,29185433,29185433,29185433
3,29185646,29185646,29185646
4,29187499,29187499,29187499
...,...,...,...
161306,182612192,182612192,182612192
161307,183366729,183366729,183366729
161308,184421950,184421950,184421950
161309,185318613,185318613,185318613
